In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from sklearn.model_selection import train_test_split

In [3]:
from qiskit_aer import AerSimulator
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister

In [5]:
import tensorflow as tf
# print(tf.__version__)
tf.config.experimental_run_functions_eagerly(True)

In [6]:
import networkx as nx

In [7]:
class Graph:
    def __init__(self, vertix=4) -> None:
        self.vertix = vertix
        self.array = np.random.randint(2, size=(self.vertix, self.vertix))
    
    def get_matrix(self):
        return self.array
    
    def get_graph(self):
        return nx.from_numpy_array(self.array)
    
    def get_chrome_number(self):
        return len(set(nx.coloring.greedy_color(self.get_graph()).values()))

In [8]:
N = 4
max_qtd = 782
epochs = 100

In [9]:
X, y = [], []
for i in range(max_qtd):
    g = Graph(N)
    X.append(g.get_matrix().flatten())
    y.append(g.get_chrome_number())

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [10]:
X_train = tf.convert_to_tensor(X_train, dtype=tf.float32)
X_test = tf.convert_to_tensor(X_test, dtype=tf.float32)
y_train = tf.convert_to_tensor(y_train, dtype=tf.float32)
y_test = tf.convert_to_tensor(y_test, dtype=tf.float32)

In [11]:
def binary2int(key):
    if not isinstance(key, str):
        key = str(key)

    return int(key, 2)



In [12]:
def bit_length(number):
    if not isinstance(number, int):
        number = int(number)

    return (number).bit_length()


In [13]:
def max_value(dic):
    max_key = max(dic, key=dic.get)

    max_value = dic[max_key]

    return max_key, max_value


In [14]:
def feature_map(inputs, bit_length_response):

    qReg = QuantumRegister(len(inputs))
        
    cReg = ClassicalRegister(bit_length_response)
    qc = QuantumCircuit(qReg, cReg)
    
    for i, x in enumerate(inputs):
        qc.rx(x, i)
    
    qc.barrier()
    
    return qc, cReg


In [15]:
def variational_circuit(qc, theta, inputs):
    
    for i in range(len(inputs) - 1):
        qc.cx(i+1, i)
    
    qc.cx(0, len(inputs)-1)
    qc.barrier()
    
    for i in range(len(inputs)):
        qc.ry(theta[i], i)

    qc.barrier()

    return qc


In [16]:
def measure(qc, bit_length_response):
    for i in range(bit_length_response):
        qc.measure(i, c[i])

    return qc


In [28]:
class QuantumLayer(tf.keras.layers.Layer):
    def __init__(self, bit_length_response, shots=1E4, **kwargs):
        super(QuantumLayer, self).__init__(**kwargs)
        self.bit_length_response = bit_length_response
        self.shots = shots

    def build(self, input_shape):
        self.N = input_shape[-1]
        self.n_response = bit_length(self.bit_length_response) # self.bit_length_response
        self.theta = self.add_weight(name='theta', shape=(self.N,), initializer='random_normal', trainable=True)

    def call(self, inputs):
        if tf.executing_eagerly():
            inputs_np = inputs.numpy()[-1]
            theta_np = self.theta.numpy()

            qc, cReg = feature_map(inputs_np, self.bit_length_response)
            qc = variational_circuit(qc, theta_np, inputs_np)
    
            for i in range(self.bit_length_response):
                qc.measure(i, cReg[i])
    
            results = AerSimulator().run(qc, shots=self.shots, memory=True).result()
            counts = results.get_counts(qc)
    
            stats = {}
            for key, value in counts.items():
                stats[binary2int(key)] = value / self.shots
    
            max_key, max_v = max_value(stats)
    
            # print(f"Key: {max_key}")

            outputs = np.zeros((tf.shape(inputs)[0], self.N))  # Modificação para retornar um tensor bidimensional
            if not max_key >= self.N:
                outputs[0, max_key] = max_v

            print(f"Max_key: {max_key}")
            outputs = tf.constant(outputs, dtype=tf.float32)

            # print(f"Outputs shape: {outputs.shape}")
            return outputs

        return inputs

    def get_config(self):
        config = super(QuantumLayer, self).get_config()
        config.update({
            'bit_length_response': self.bit_length_response,
            'shots': self.shots,
        })
        return config


In [32]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(N, activation=tf.nn.relu, name="Dense_1"),
    QuantumLayer(N, name="QuantumLayer"),
    tf.keras.layers.Dense(N+1, activation='softmax', name="Dense_2")
])
model.build()

In [33]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, epochs=epochs, validation_data=(X_test, y_test))

Epoch 1/100
Max_key: 0
 1/17 ━━━━━━━━━━━━━━━━━━━━ 2s 152ms/step - accuracy: 0.0312 - loss: 1.5952Max_key: 0
Max_key: 9
 3/17 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.3194 - loss: 1.5961 Max_key: 0
Max_key: 0
Max_key: 0
 6/17 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.4359 - loss: 1.5984

/usr/local/lib/python3.12/site-packages/keras/src/optimizers/base_optimizer.py:576: UserWarning: Gradients do not exist for variables ['kernel', 'bias', 'theta'] when minimizing the loss. If using `model.compile()`, did you forget to provide a `loss` argument?
  warnings.warn(


Max_key: 0
Max_key: 0
 8/17 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.4653 - loss: 1.5990Max_key: 9
Max_key: 9
10/17 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.4847 - loss: 1.5993Max_key: 0
Max_key: 0
12/17 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.4997 - loss: 1.5995Max_key: 9
Max_key: 0
Max_key: 0
15/17 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.5175 - loss: 1.5996Max_key: 0
Max_key: 0
Max_key: 0
Max_key: 0
Max_key: 0
Max_key: 0
Max_key: 0
Max_key: 0
Max_key: 0
Max_key: 0
Max_key: 0
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - accuracy: 0.5317 - loss: 1.5989 - val_accuracy: 0.6486 - val_loss: 1.5675
Epoch 2/100
Max_key: 0
 1/17 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.6562 - loss: 1.5986Max_key: 0
Max_key: 0
 3/17 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.6250 - loss: 1.5934Max_key: 0
Max_key: 0
 5/17 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.6206 - loss: 1.5919Max_key: 0
Max_key: 0
Max_key: 0
 8/17 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0

In [ ]:
train_accuracy = history.history['accuracy']
test_accuracy = history.history['val_accuracy']

In [ ]:
plt.plot(range(1, len(train_accuracy) + 1), train_accuracy, label='Acurácia de Treino')
plt.plot(range(1, len(test_accuracy) + 1), test_accuracy, label='Acurácia de Teste')
plt.xlabel('Épocas')
plt.ylabel('Acurácia')
plt.title('Acurácia ao longo das Épocas')
plt.legend()
plt.show()